In [48]:
import numpy as np
import pandas as pd
import haversine
import math

In [2]:
fn = "../data/deped_school_location_with_enrolment_2017_2018_geocoded1_intersection.csv"
df = pd.read_csv(fn)

In [34]:
# read in raw dataset
fn2 = "../data/deped_school_location_with_enrolment_2017_2018.csv"
df2 = pd.read_csv(fn2)

In [36]:
fn_out = "../data/deped_school_location_with_enrolment_2017_2018_geocoded1_validationB.csv"

In [35]:
len(df), len(df2)

(5930, 47358)

In [6]:
df.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'SCHOOL NAME2', u'ADDRESS1', u'ADDRESS2', u'GMAP_MAPPED', u'GMAP_LAT1',
       u'GMAP_LON1', u'GMAP_LAT2', u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code',
       u'Reg_Name', u'Pro_Code', u'Pro_Name', u'Mun_Code', u'Mun_Name',
       u'Bgy_Code', u'Bgy_Name'],
      dtype='object')

In [8]:
df.head(2)

,SCHOOL ID,SCHOOL NAME,ADDRESS,REGION,DIVISION,MUNICIPALITY,BARANGGAY,LATITUDE,LONGITUDE,MAPPED,...,GMAP_LON2,GMAP_DELTA_KM,Reg_Code,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name
0,500000,CARAITAN INTEGRATED SCHOOL,"CARAITAN, BADOC, ILOCOS NORTE, BRGY. CARAITAN,...",REGION I,ILOCOS NORTE,BADOC,CARAITAN,NaN,NaN,0,...,120.487604,0.000000,PH010000000,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012803000,BADOC,PH012803009,Caraitan
1,500594,UGUIS INTEGRATED SCHOOL,"BRGY. 9, UGUIS, BRGY. UGUIS, NUEVA ERA",REGION I,ILOCOS NORTE,NUEVA ERA,UGUIS,NaN,NaN,0,...,120.575476,20.160737,PH010000000,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012814000,NUEVA ERA,PH012814010,Santo Niño


In [9]:
# convert to upper case
for col in "REGION", "MUNICIPALITY", "BARANGGAY", "Bgy_Name", "Mun_Name", "Reg_Name":
    df[col] = df[col].str.upper()

In [10]:
# compare strings - check if one is a substring of the other
df["MATCH_BARANGAY"] = np.nan
df["MATCH_MUNICIPALITY"] = np.nan
df["MATCH_REGION"] = np.nan
col1 = ["Bgy_Name", "Mun_Name", "Reg_Name"]
col2 = ["BARANGGAY", "MUNICIPALITY", "REGION"]
col_out = ["MATCH_BARANGAY", "MATCH_MUNICIPALITY", "MATCH_REGION"]
for i in np.arange(len(df)):
    row = df.loc[i]
    for j in np.arange(3):
        str1, str2 = row[col1[j]], row[col2[j]]
        if (str1 in str2) or (str2 in str1):
            match = 1
        else:
            match = 0
        df.loc[i, col_out[j]] = match

In [11]:
df.head()

,SCHOOL ID,SCHOOL NAME,ADDRESS,REGION,DIVISION,MUNICIPALITY,BARANGGAY,LATITUDE,LONGITUDE,MAPPED,...,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,MATCH_BARANGAY,MATCH_MUNICIPALITY,MATCH_REGION
0,500000,CARAITAN INTEGRATED SCHOOL,"CARAITAN, BADOC, ILOCOS NORTE, BRGY. CARAITAN,...",REGION I,ILOCOS NORTE,BADOC,CARAITAN,NaN,NaN,0,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012803000,BADOC,PH012803009,CARAITAN,1.0,1.0,1.0
1,500594,UGUIS INTEGRATED SCHOOL,"BRGY. 9, UGUIS, BRGY. UGUIS, NUEVA ERA",REGION I,ILOCOS NORTE,NUEVA ERA,UGUIS,NaN,NaN,0,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012814000,NUEVA ERA,PH012814010,SANTO NIñO,0.0,1.0,1.0
2,500345,MALAGUIP INTEGRATED SCHOOL,"MASINTOC, PAOAY, ILOCOS NORTE, BRGY. MASINTOC,...",REGION I,ILOCOS NORTE,PAOAY,MASINTOC,NaN,NaN,0,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012816000,PAOAY,PH012816008,MASINTOC,1.0,1.0,1.0
3,100185,NAGBACALAN ES,"NAGBACALAN, PAOAY",REGION I,ILOCOS NORTE,PAOAY,NAGBACALAN,NaN,NaN,0,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012816000,PAOAY,PH012816011,NAGBACALAN,1.0,1.0,1.0
4,500593,SACRITAN IS,"BRGY. BALBALAY, SACRITAN, BRGY. SACRITAN, PINILI",REGION I,ILOCOS NORTE,PINILI,SACRITAN,NaN,NaN,0,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012814000,NUEVA ERA,PH012814007,GARNADEN,0.0,0.0,1.0


In [18]:
len(df), len(df)*1./6144

(5930, 0.9651692708333334)

In [20]:
df["MAPPED"].value_counts()

0    4959
1     971
Name: MAPPED, dtype: int64

In [28]:
4959./5144.

0.9640357698289269

In [23]:
# define INVALID = 1, if both municipality and region does not match
df["INVALID"] = 0
df.loc[(df[col_out[2]]==0) & (df[col_out[1]]==0), "INVALID"] = 1

In [24]:
dfM0 = df[df["MAPPED"]==1] # baseline
dfM1 = df[df["MAPPED"]==0]

In [25]:
dfM0["INVALID"].value_counts()

0    913
1     58
Name: INVALID, dtype: int64

In [27]:
58./971

0.05973223480947477

In [26]:
dfM1["INVALID"].value_counts()

0    4713
1     246
Name: INVALID, dtype: int64

In [29]:
246./4959

0.049606775559588624

In [12]:
df[col_out[0]].value_counts(1)

0.0    0.547049
1.0    0.452951
Name: MATCH_BARANGAY, dtype: float64

In [13]:
df[col_out[1]].value_counts(1)

1.0    0.860034
0.0    0.139966
Name: MATCH_MUNICIPALITY, dtype: float64

In [14]:
df[col_out[2]].value_counts(1)

1.0    0.926644
0.0    0.073356
Name: MATCH_REGION, dtype: float64

In [30]:
df_m2 = df[df[col_out[2]]==0]
len(df_m2)

435

In [31]:
df_m1 = df[(df[col_out[2]]==0) & (df[col_out[1]]==0)]
len(df_m1)

304

In [32]:
df_m1

,SCHOOL ID,SCHOOL NAME,ADDRESS,REGION,DIVISION,MUNICIPALITY,BARANGGAY,LATITUDE,LONGITUDE,MAPPED,...,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,MATCH_BARANGAY,MATCH_MUNICIPALITY,MATCH_REGION,INVALID
9,107421,TAM-AWAN PRIMARY SCHOOL,"ALULING, CERVANTES, ILOCOS SUR, BRGY. ALULING,...",REGION I,ILOCOS SUR,CERVANTES,ALULING,NaN,NaN,0,...,PH141100000,BENGUET,PH141102000,BAGUIO CITY,PH141102088,PINSAO PROPER,0.0,0.0,0.0,1
121,500004,ESPERANZA INTEGRATED SCHOOL,"ZONE III BARANGAY ESPERANZA, BRGY. ESPERANZA, ...",REGION I,"PANGASINAN II, BINALONAN",SISON,ESPERANZA,NaN,NaN,0,...,PH086000000,SAMAR (WESTERN SAMAR),PH086003000,CALBAYOG CITY,PH086003156,VILLAHERMOSA,0.0,0.0,0.0,1
181,103003,STO. DOMINGO ELEMENTARY SCHOOL,"NABOR ST., BRGY. SANTO DOMINGO, ALICIA",REGION II,ISABELA,ALICIA,SANTO DOMINGO,NaN,NaN,0,...,PH043400000,LAGUNA,PH043402000,BAY,PH043402013,SANTO DOMINGO,1.0,0.0,0.0,1
197,155518,DAGUPAN ELEMENTARY SCHOOL,"PUROK 3, BRGY. DAGUPAN, BENITO SOLIVEN",REGION II,ISABELA,BENITO SOLIVEN,DAGUPAN,NaN,NaN,0,...,PH015500000,PANGASINAN,PH015518000,DAGUPAN CITY,PH015518024,BARANGAY I (T. BUGALLON),0.0,0.0,0.0,1
237,103284,BENGUET ELEMENTARY SCHOOL,"PUROK 4 BENGUET ECHAGUE, BRGY. BENGUET, ECHAGUE",REGION II,ISABELA,ECHAGUE,BENGUET,NaN,NaN,0,...,PH141100000,BENGUET,PH141104000,BOKOD,PH141104002,BILA,0.0,0.0,0.0,1
241,103286,CASTILLO ELEMENTARY SCHOOL,"PRK.3 NATIONAL HIGHWAY, BRGY. CASTILLO, ECHAGUE",REGION II,ISABELA,ECHAGUE,CASTILLO,NaN,NaN,0,...,PH041000000,BATANGAS,PH041020000,PADRE GARCIA,PH041020016,CASTILLO,1.0,0.0,0.0,1
271,103319,SAN FELIPE ELEMENTARY SCHOOL,"PUROK 04, BRGY. SAN FELIPE, ECHAGUE",REGION II,ISABELA,ECHAGUE,SAN FELIPE,NaN,NaN,0,...,PH041000000,BATANGAS,PH041020000,PADRE GARCIA,PH041020026,SAN FELIPE,1.0,0.0,0.0,1
285,500970,STO. DOMINGO-SAN ISIDRO INTEGRATED SCHOOL,"GALAM ST., BRGY. SANTO DOMINGO",REGION II,ISABELA,LUNA,SANTO DOMINGO,NaN,NaN,0,...,PH050500000,ALBAY,PH050516000,SANTO DOMINGO (LIBOG),PH050516021,SAN ISIDRO,0.0,0.0,0.0,1
293,500467,STO. TOMAS INTEGRATED SCHOOL,"STO. TOMAS, NAGUILIAN, BRGY. SANTO TOMAS, NAGU...",REGION II,ISABELA,NAGUILIAN,SANTO TOMAS,NaN,NaN,0,...,PH041000000,BATANGAS,PH041028000,SANTO TOMAS,PH041028022,SAN VICENTE,0.0,0.0,0.0,1
313,103562,ABUT ES,"-, BRGY. ABUT",REGION II,ISABELA,QUEZON,ABUT,NaN,NaN,0,...,PH051700000,CAMARINES SUR,PH051721000,MILAOR,PH051721015,SAN JOSE (POB.),0.0,0.0,0.0,1


In [33]:
len(df[df["INVALID"]==1]), len(df[df["INVALID"]==1])*1./len(df)

(304, 0.051264755480607084)

In [ ]:
# merge with raw dataset 

In [42]:
df_out = pd.merge(df2, df[[u'SCHOOL ID', 'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1',
       u'GMAP_LON1', u'GMAP_LAT2', u'GMAP_LON2', u'GMAP_DELTA_KM',
       u'Reg_Code', u'Reg_Name', u'Pro_Code', u'Pro_Name', u'Mun_Code',
       u'Mun_Name', u'Bgy_Code', u'Bgy_Name', "MATCH_BARANGAY", "MATCH_MUNICIPALITY", "MATCH_REGION"]], on="SCHOOL ID", how="left")

In [44]:
df_out.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code', u'Reg_Name', u'Pro_Code',
       u'Pro_Name', u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name',
       u'MATCH_BARANGAY', u'MATCH_MUNICIPALITY', u'MATCH_REGION'],
      dtype='object')

In [71]:
df_out.to_csv(fn_out, index=False)